<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Sampling Methods Demonstration


### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)


### The Interactive Workflow

Here's a simple workflow for comparing random and orthogonal sampling. 

* we use a 'toy problem' to demonstrate and compare these sampling methods 

#### Sampling

While statistical theory supports random sampling, the fluctuation in sample statistics is quite extreme for small samples sizes. For example, the variance in the sample mean is calculated by standard error:

\begin{equation}
\sigma_{\overline{x}}^2 = \frac{\sigma_s^2}{n}
\end{equation}

To suppress these statistical fluctuations, alternative sampling methods are available:

1. **Random Sampling** - next sample is drawn without consideration of the previously drawn samples
2. **Latin Hypercube Sampling** - apply $k$ equiprobability bins to each feature, $X_m^k, m=1,...,M$. Then draw one sample from each bin, $n(X_m^k)=1$. 
3. **Orthogonal Sampling** - divide the joint probability density function into $k$ equal probability subspaces and then randomly draw an equal number of samples, $\frac{n}{k}$ from each subspace. 

#### Objective 

An interactive exercise to try out and compare random and orthgonal sampling.

* observe the stabilization of the sample statistics
* observe the impact of number of regions on the results for orthogonal sampling

#### Getting Started

Here's the steps to get setup in Python with the GeostatsPy package:

1. Install Anaconda 3 on your machine (https://www.anaconda.com/download/). 
2. From Anaconda Navigator (within Anaconda3 group), go to the environment tab, click on base (root) green arrow and open a terminal. 
3. In the terminal type: pip install geostatspy. 
4. Open Jupyter and in the top block get started by copy and pasting the code block below from this Jupyter Notebook to start using the geostatspy functionality. 

You will need to copy the data file to your working directory.  They are available here:

* Tabular data - sample_data.csv at https://git.io/fh4gm.

There are exampled below with these functions. You can go here to see a list of the available functions, https://git.io/fh4eX, other example workflows and source code. 

#### Load the required libraries

The following code loads the required libraries.

In [6]:
import numpy as np                                      # arrays and array math
import pandas as pd                                     # tabular data and tabular data math
import matplotlib.pyplot as plt                         # data visualization
from matplotlib.cm import colors              
import scipy.stats as stats                             # Gaussian PDF and random sampling
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox

#### Interactive Sampling Methods

The following code includes:

* dashboard with data and orthogonal sampling parameters, number of samples and number of subspaces

* plots of the data distribution and random and orthogonal samples

In [49]:
# interactive calculation of the sample set (control of source parametric distribution and number of samples)
style = {'description_width': 'initial'}
l = widgets.Text(value='                                              Sampling Methods, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
nsamp = widgets.IntSlider(min = 1, max = 1000, value = 10, step = 5, description = '$n_{sample}$',orientation='horizontal',
                          layout=Layout(width='500px', height='30px'),continuous_update = False)
nsamp.style.handle_color = 'darkorange'
npart = widgets.IntSlider(min = 1, max = 20, value = 4, step = 1, description = '$n_{subspace}$',orientation='horizontal',
                          layout=Layout(width='500px', height='30px'),continuous_update = False)
npart.style.handle_color = 'darkorange'

uipars = widgets.HBox([nsamp,npart],) 
uik = widgets.VBox([l,uipars],)

def f_make_sample(nsamp,npart): # function to take parameters, make sample and plot
    mean = 10.0; stdev = 2.0
    npop = 100000
    parts = []
    np.random.seed(seed = 79079)
    nbin = 70
    hbins = np.linspace(0,20,nbin)
    shbins = np.linspace(0,20,nbin*100)
    
    cmap = plt.cm.hot; norm = colors.Normalize(vmin=1, vmax=npart+1)
    
    x = np.random.normal(loc=mean,scale=stdev,size=npop)
    xs = np.random.choice(x,nsamp,replace = False)
    yhat = stats.norm.pdf(shbins,loc=mean,scale=stdev)
    
    ax1 = plt.subplot(121)
    ax1.plot(shbins,yhat,color='black',lw=2.0,zorder=1)
    
    ax2 = ax1.twinx()
    hist2ax,_,_ = ax2.hist(xs,bins=hbins,color='grey',alpha=1.0,edgecolor='black',zorder=10,density=True,
                           histtype=u'step',linewidth=2,label='Samples'); ax1.set_xlabel('Porosity (%)')
    ax2.hist(xs,bins=hbins,color='grey',alpha=0.2,zorder=20,density=True)
    ax1.fill_between(shbins,0,yhat,color='darkorange',alpha=0.8,zorder=1)
    ax1.set_xlabel('Porosity (%)'); ax1.set_ylabel('Population Density'); ax1.set_title('Population and Random Sample'); 
    ax1.set_ylim([0,0.3]); ax1.set_xlim([2,18])
    plt.legend(loc='upper right')
    ax2.set_ylabel('Sample Density',rotation=270,labelpad=20);
    
    pbins = np.percentile(x,np.linspace(0,100,npart+1))
    int_values = pd.cut(x, pbins,labels = np.arange(1,npart+1,1))
    
    for i in range(0,npart):
        parts.append(x[int_values == int(i+1)])
    
    latin_samples = np.zeros(nsamp)
    ipart = 0
    for isamp in range(0,nsamp):
        latin_samples[isamp] = np.random.choice(parts[ipart],1,replace = False) 
        ipart = ipart + 1
        if ipart >= npart:
            ipart = 0
            
    ax3 = plt.subplot(122)
    ax3.plot(shbins,yhat,color='black',lw=2.0,zorder=1)
    
    ax4 = ax3.twinx()
    hist2bx,_,_ = ax4.hist(latin_samples,bins=hbins,color='grey',alpha=1.0,edgecolor='black',zorder=10,density=True,
                           histtype=u'step',linewidth=2,label='Samples')
    ax4.hist(latin_samples,bins=hbins,color='grey',alpha=0.2,zorder=20,density=True)  
    ax3.set_xlabel('Porosity (%)'); ax3.set_title('Population and Orthogonal Samples')
    ax3.set_ylabel('Population Density'); ax3.set_ylim([0,0.3]); ax3.set_xlim([2,18])
    plt.legend(loc='upper right'); ax4.set_ylabel('Sample Density',rotation=270,labelpad=20)
    
    i = 0
    for fbin in pbins[1:]:
        ax3.vlines(fbin,0,stats.norm.pdf(fbin,loc=mean,scale=stdev),color='black',lw=1.0)
        ax3.fill_between(shbins,0,yhat,color=plt.cm.inferno(i/(npart+1)),alpha=0.8,where=(np.logical_and([shbins > pbins[i]],[shbins < pbins[i+1]])[0]),zorder=1)
        i = i + 1
       
    ylim_24 = max(np.max(hist2ax)*(1.1 + (1.25-1.1)/1000 * nsamp),np.max(hist2bx)*(1.1 + (1.25-1.1)/1000 * nsamp))
    ax2.set_ylim([0.0,ylim_24])
    ax4.set_ylim([0.0,ylim_24])
        
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.3, hspace=0.2); plt.show()
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(f_make_sample, {'nsamp':nsamp, 'npart':npart})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

### Interactive Sampling Demonstration

Compare random and orthogonal sampling. Select the number of samples and number of subspaces and observe the sample distribution.

#### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

### The Inputs

* **$n_{sample}$** - the number of samples, **$n_{subspace}$** - the number of orthogonal subspaces

In [50]:
display(uik, interactive_plot)                            # display the interactive plot

Output()

#### Comments

This was an interactive demonstration of sampling for data analytics. Much more could be done, I have other demonstrations on the basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 
  
#### The Author:

### Michael Pyrcz, Associate Professor, University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) 